<div style="border-radius: 10px; border: #DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size: 100%; text-align: left">
<h3 align="left"><font color='#3498DB'> statsmodels: 武者修行</font></h3>

このノートブックは個人的な学習のために作られたものです。  
Pythonしか使えない状況で時系列分析を行いたい場合はstatsmodelsを使うのがよいと考えています。  
以下では、statsmodelsのUser Guid(https://www.statsmodels.org/stable/user-guide.html#background)を引用しています。  
一部のUser Guidを抜き出してChatGPTで翻訳しています。

### はじめに
statsmodelsでは、データの要素と説明変数を指すためにendogとexogという名前を使用しています。これらは、推定問題で使用される観測変数です。

### 背景

これらの用語の非公式な定義のいくつかは次のようになります。

endogenous：システム内の要因によって引き起こされる

exogenous：システム外の要因によって引き起こされる

内生変数は、そのモデルによって説明または予測される経済/計量モデルの変数を指します。

外生変数は、経済/計量モデルに現れる変数を指しますが、そのモデルによって説明されない（つまり、モデルによって与えられたものとして取り扱われる）変数を指します。

計量経済学と統計学では、モデルに応じて異なる形式の外生性（弱い、強い、厳格な）の定義があります。statsmodelsでの変数名としての使用は常に厳密な意味で解釈できるわけではありませんが、同じ原則に従おうとしています。

最も単純な形では、モデルは観測変数yを、他の一連の変数xにいくつかの線形または非線形形式で関連付けます。

y = f(x, beta) + ノイズ  
y = x * beta + ノイズ

ただし、統計モデルを持つには、説明変数xとノイズの性質に関する追加の仮定が必要です。多くの基本モデルに対する1つの標準的な仮定は、xがノイズと相関していないというものです。より一般的な定義では、xが外生であることは、説明変数xの生成方法を考慮しないで、xがyに対してどのような効果や影響を推定したり、その効果に関する仮説を検定したりする必要がないことを意味します。

言い換えれば、yは私たちのモデルに対して内生的であり、xは私たちのモデルに対して外生的であることを意味します。

例えば、実験を実施し、2回目のセッションに一部の被験者が利用できなくなったとしましょう。そのドロップアウトは実験の結論にとって関連があるでしょうか？言い換えれば、ドロップアウトの決定を私たちの問題に対して外生的に扱うことができるでしょうか。

ユーザーは、モデルの基礎的な統計的仮定が何であるかを知るか、テキストブックを参照するかして、モデルの基礎的な統計的仮定が何であるかを知るか、テキストブックを参照する必要があります。例えば、OLSの場合、エラーまたはノイズ項が時間に独立に分布している場合、エクソグでは遅延した従属変数を持つことができます。しかし、エラー項がラグ付きの従属変数と自己相関している場合、OLSは良い統計的性質を持たず（不一致性がある）、正しいモデルはARMAXになります。statsmodelsには、統計的仮定が正当化されているかどうかを検証するための回帰診断の機能があります。

### Fitting models using R-style formulas

バージョン0.5.0以降、statsmodelsではRスタイルの式を使用して統計モデルを適合させることができます。内部的には、式とデータをモデル適合に使用される行列に変換するためにpatsyパッケージを使用しています。式フレームワークは非常に強力で、このチュートリアルでは基本的な部分しか触れません。式言語の詳細な説明は、patsyのドキュメントで見つけることができます。

モジュールと関数の読み込み

```python
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
import pandas
```

通常のstatsmodels.apiに加えて、式のモデルを扱うためにstatsmodels.formula.apiを呼び出しています。実際、ここではデータセットを読み込むためにだけstatsmodels.apiが使用されています。formula.apiには、apiで見られる関数の多くが含まれています（例：OLS、GLMなど）、ただし、これらのモデルのほとんどに小文字の対応関数も含まれています。一般的に、小文字のモデルは式とdf引数を受け入れ、大文字のモデルはendogとexogデザイン行列を取ります。formulaはpandasデータフレームを取ります。

`dir(smf)` は利用可能なモデルのリストを表示します。

式と予測変数を使用するモデルの一般的な呼び出しシグネチャは次のようになります：(formula, data, subset=None, *args, **kwargs)

式を使用したOLS回帰

まず、Getting Startedページで説明されている線形モデルを適合させます。データをダウンロードし、列を選択し、欠損観測を削除します。

```python
df = sm.datasets.get_rdataset("Guerry", "HistData").data
df = df[['Lottery', 'Literacy', 'Wealth', 'Region']].dropna()
```

モデルを適合させます。

```python
mod = smf.ols(formula='Lottery ~ Literacy + Wealth + Region', data=df)
res = mod.fit()
print(res.summary())
```

この例では、LotteryをLiteracyとWealth、Regionに関連付けるモデルをOLS回帰で適合させています。

カテゴリカル変数

上記で表示されたサマリーを見ると、patsyはRegionの要素をテキスト文字列として認識し、Regionをカテゴリカル変数として扱いました。patsyのデフォルトは、切片を含むことですので、自動的にRegionのカテゴリーの1つを削除しました。

もしRegionが整数変数であり、それを明示的にカテゴリカルとして扱いたい場合、C()演算子を使用できます。

```python
res = smf.ols(formula='Lottery ~ Literacy + Wealth + C(Region)', data=df).fit()
print(res.params)
```

演算子

すでに、"~"はモデルの左辺と右辺を区別し、"+"はデザイン行列に新しい列を追加することを示していることを見ました。

変数の削除

"-"記号を使用して列/変数を削除することができます。例えば、モデルから切片を削除するには次のようにします。

```python
res = smf.ols(formula='Lottery ~ Literacy + Wealth + C(Region) - 1 ', data=df).fit()
print(res.params)
```

乗算的な相互作用

":"は他の2つの列の積を含む新しい列をデザイン行列に追加します。"*"は積を含む個々の列も含めます。

```python
res1 = smf.ols(formula='Lottery ~ Literacy : Wealth - 1', data=df).fit()
res2 = smf.ols(formula='Lottery ~ Literacy * Wealth - 1', data=df).fit()
print(res1.params)
print(res2.params)
```

関数

モデルの変数にベクトル化された関数を適用できます。

```python
res = smf.ols(formula='Lottery ~ np.log(Literacy)', data=df).fit()
print(res.params)
```

カスタム関数の定義

カスタム関数を定義できます。

```python
def log_plus_1(x):
    return np.log(x) + 1.0

res = smf.ols(formula='Lottery ~ log_plus_1(Literacy)', data=df).fit()
print(res.params)
```

名前空間

上記のすべての例では、関数を適用するために呼び出し元の名前空間を使用しています。名前空間はeval_envキーワードを使用して制御できます。たとえば、patsy:patsy.EvalEnvironmentを使用してカスタム名前空間を指定したり、eval_func=-1を渡すことで「クリーン」な名前空間を使用したりできます。デフォルトは呼び出し元の名前空間を使用することです。これは、たとえば、誰かがユーザー名前空間またはpatsyに渡されるデータ構造内に変数Cがある場合、Cがカテゴリカル変

数を処理するための式で使用されている場合に（予期されない）結果をもたらすことがあります。詳細についてはPatsy APIリファレンスを参照してください。

式をサポートしない（またはまだサポートしていない）モデルで式を使用する

特定のstatsmodels関数が式をサポートしていない場合でも、式言語を使用してデザイン行列を生成し、それらの行列を適合関数にendogとexog引数として提供できます。

numpy配列を生成するには：

```python
import patsy
f = 'Lottery ~ Literacy * Wealth'
y, X = patsy.dmatrices(f, df, return_type='matrix')
```

pandasデータフレームを生成するには：

```python
f = 'Lottery ~ Literacy * Wealth'
y, X = patsy.dmatrices(f, df, return_type='dataframe')
```

yとXはpatsy.DesignMatrixのインスタンスで、numpy.ndarrayのサブクラスです。

Pandasデータフレームを生成するには：

```python
In [29]: f = 'Lottery ~ Literacy * Wealth'

In [30]: y, X = patsy.dmatrices(f, df, return_type='dataframe')

In [31]: print(y[:5])
   Lottery
0     41.0
1     38.0
2     66.0
3     80.0
4     79.0

In [32]: print(X[:5])
   Intercept  Literacy  Wealth  Literacy:Wealth
0        1.0      37.0    73.0           2701.0
1        1.0      51.0    22.0           1122.0
2        1.0      13.0    61.0            793.0
3        1.0      46.0    76.0           3496.0
4        1.0      69.0    83.0           5727.0
```

[33] のコードでは、データフレームdfから、Lotteryを目的変数とし、LiteracyとWealth、LiteracyとWealthの積を説明変数とする線形回帰モデルをOLS回帰で適合させています。そして、結果のサマリーを表示しています。

### Pitfall

このページでは、statsmodelsを使用する際に発生するかもしれない問題をリストしています。これらの問題は、データに関連する問題、統計的な問題、ソフトウェアの設計、モデルの「非標準的な」使用、またはエッジケースの結果である可能性があります。

statsmodelsは診断チェック用の警告とヘルパー関数を提供しており（線形回帰のミススペシフィケーションチェックの例は、このブログ記事を参照してください）、カバレッジはもちろん包括的ではありませんが、時間の経過とともにより多くの警告と診断関数が追加されます。

基本的な統計的問題はすべての統計パッケージに共通ですが、ソフトウェアの実装は極端なケースや特異なケースの処理方法が異なります。モデルが動作しない可能性がある特異なケースを報告していただければ、適切に処理できるようになります。
異なるパラメータでの繰り返し適合呼び出し¶

結果のインスタンスは、しばしば対応するモデルインスタンスから属性にアクセスする必要があります。異なる引数を使用してモデルを複数回適合させることは、モデルの属性を変更できます。これは、モデルを再適合した後、結果のインスタンスが正しいモデルの属性を指さなくなる可能性があることを意味します。

したがって、異なる適合関数引数を使用してモデルを適合させたい場合には、別々のモデルインスタンスを作成するのがベストプラクティスです。

たとえば、以下の例では問題ありません。なぜなら、結果のインスタンスをさらに使用しないためです。

```python
mod = AR(endog)
aic = []
for lag in range(1,11):
    res = mod.fit(maxlag=lag)
    aic.append(res.aic)
```

しかし、異なる2つの異なる推定結果を保持したい場合、2つの別々のモデルインスタンスを作成することをお勧めします。

```python
mod1 = RLM(endog, exog)
res1 = mod1.fit(scale_est='mad')
mod2 = RLM(endog, exog)
res2 = mod2.fit(scale_est=sm.robust.scale.HuberScale())
```

未識別のパラメータ¶
ランク不足のexog、完璧な多重共線性¶

線形モデル、GLS、RLM、GLMなどのモデルは一般化逆行列を使用します。これは次のことを意味します：

- ランク不足の行列はエラーを発生させません
- ほぼ完全な多重共線性または病的な設計行列の場合、数値的に安定しない結果が発生するかもしれません。これは、ユーザーが行列のランクや条件数を手動で確認する必要がある場合です。

注意：NISTのFilipのベンチマークケースについて、データをスケーリングしない場合、statsmodelsは現在失敗します。詳細については、このブログを参照してください。

最尤推定の収束不足¶

いくつかのケースでは、最尤推定値が存在せず、パラメータが無限であるか、一意ではないかもしれません（例：バイナリの従属変数を持つモデルでの（準）分離）。デフォルトの設定では、statsmodelsは最適化アルゴリズムが収束しない場合に警告を表示します。ただし、収束基準は時々誤って収束を示すことがあります（目的関数の値が収束したがパラメータは収束しなかった場合など）。一般的には、ユーザーは収束を確認する必要があります。

バイナリのLogitおよびProbitモデルの場合、statsmodelsは完璧な予測が検出された場合に例外を発生させます。ただし、ほぼ完璧な予測についてはチェックされていません。
その他の問題¶
データの変動不足¶

小さなデータセットやカテゴリカル変数の小さなグループを持つデータの場合、データの変動が不足している可能性があります。これらの場合、結果は識別されないか、いくつかの隠れた問題が発生する可能性があります。

現在知られている唯一のケースは、頑健な線形モデルの推定において完全な適合です

。RLMの場合、残差がゼロの場合、例外を発生させませんが、この完全な適合はいくつかの結果でNaN（scale=0および0/0の除算）を生成する可能性があります（問題＃55）。
モデルのドメイン外の真のパラメータ¶

一部の場合、モデル内のパラメータのドメイン制約がデータと矛盾するかもしれません。これらの場合、推定はパラメータスペースの境界に近いパラメータ値で停止するかもしれませんが、最適化中にランタイムエラーが発生するか、最適化中にnansが生成される可能性があります。

2つの例：

1. データがポアソンと同じ分散を持つか、過分散されていない場合、つまり真のモデルがポアソンと同じ分散を持つか過分散されていない場合、ネガティブバイナミアルモデルの推定は一貫性がありません。statsmodelsで実装されているように、分散パラメータの値がゼロ分散またはゼロ分散の近くで評価できないため、loglikelihoodおよびその導関数は評価できません。

2. ゼロ膨張モデルは現在、膨張モデルのモデルとしてLogitまたはProbitを使用します。これは、膨張がパラメータスペースの境界にあることを意味し、ゼロ収縮が有効なパラメータスペースの外にあることを意味します。データにゼロの膨張がないか、ゼロの収縮がない場合、ゼロ膨張モデルは最適化に失敗するか、無膨張の境界に近くなることがよくあります。

## Time Series（時系列）の分析 tsa¶

statsmodels.tsaには、時系列分析に役立つモデルクラスと関数が含まれています。基本的なモデルには、単変量自己回帰モデル（AR）、ベクトル自己回帰モデル（VAR）、単変量自己回帰移動平均モデル（ARMA）が含まれます。非線形モデルには、マルコフ・スイッチング・ダイナミック・リグレッションや自己回帰なども含まれます。また、時系列の記述統計情報も含まれており、自己相関、偏自己相関関数、パワースペクトル密度などが含まれます。また、ARMAや関連プロセスの理論的な特性も含まれています。さらに、自己回帰および移動平均のラグ多項式と連携するためのメソッドも含まれています。関連する統計テストと便利なヘルパー関数も利用可能です。

推定は、厳密または条件つき最尤法または条件つき最小二乗法を使用して、カルマンフィルターまたは直接フィルターを使用して行われます。

現在、関数とクラスは対応するモジュールからインポートする必要がありますが、主要なクラスはstatsmodels.tsa名前空間で利用できるようになります。モジュールの構造はstatsmodels.tsa内にあります。

- stattools：経験的な特性とテスト、自己相関関数（acf）、偏自己相関関数（pacf）、Granger因果関係、ADF単位根テスト、KPSSテスト、BDSテスト、Ljung-Boxテストなど。

- ar_model：単変量自己回帰プロセス、条件および厳密な最尤法、条件つき最小二乗法による推定

- arima.model：単変量ARIMAプロセス、代替手法による推定

- statespace：包括的な状態空間モデルの仕様と推定。状態空間のドキュメンテーションを参照してください。

- vector_ar, var：ベクトル自己回帰プロセス（VAR）およびベクトル誤差修正モデル、推定、インパルス応答解析、予測誤差分散分解、データ可視化ツール。ベクトル_arのドキュメンテーションを参照してください。

- arma_process：指定されたパラメータを持つARMAプロセスの特性。これにはARMA、MA、AR表現間の変換、自己相関関数（acf）、偏自己相関関数（pacf）、スペクトル密度、インパルス応答関数などのツールが含まれています。

- sandbox.tsa.fftarma：arma_processに類似しており、周波数領域で動作します。

- tsatools：トレンドのためのラグ変数の配列を作成し、トレンドの回帰子を構築するなど、追加のヘルパー関数

- filters：時系列のフィルタリング用のヘルパー関数

- regime_switching：マルコフ・スイッチング・ダイナミック・リグレッションおよび自己回帰モデル

時系列分析に役立つ他の一部の関数は、statsmodelsの他の部分にもあります。関連する関数はまた、matplotlib、nitime、およびscikits.talkboxでも利用可能です。これらの関数は通常、長期の時系列が利用可能で、より頻繁に周波数領域で動作する信号処理に使用されることを設計されています。

| 関数                                                | 説明                                                                     |
|---------------------------------------------------|--------------------------------------------------------------------------|
| `stattools.acovf(x[, adjusted, demean, fft, ...])` | 自己共分散を推定します。                                                |
| `stattools.acf(x[, adjusted, nlags, qstat, ...])`  | 自己相関関数を計算します。                                             |
| `stattools.pacf(x[, nlags, method, alpha])`       | 偏自己相関を推定します。                                               |
| `stattools.pacf_yw(x[, nlags, method])`          | 非再帰のYule-Walker法を使用して推定された偏自己相関。                 |
| `stattools.pacf_ols(x[, nlags, efficient, ...])`  | OLS（最小二乗法）を使用して偏自己相関を計算します。                  |
| `stattools.pacf_burg(x[, nlags, demean])`        | Burg法を使用して偏自己相関を計算します。                               |
| `stattools.ccovf(x, y[, adjusted, demean, fft])` | 2つの系列間の相互共分散を計算します。                                   |
| `stattools.ccf(x, y[, adjusted, fft])`           | 相互相関関数。                                                        |
| `stattools.adfuller(x[, maxlag, regression, ...])`| Augmented Dickey-Fuller単位根検定。                                     |
| `stattools.kpss(x[, regression, nlags, store])`   | 定常性のためのKwiatkowski-Phillips-Schmidt-Shin検定。                   |
| `stattools.range_unit_root_test(x[, store])`     | 定常性のための範囲単位根検定。                                          |
| `stattools.zivot_andrews`                        | Zivot-Andrews構造変更単位根検定。                                       |
| `stattools.coint(y0, y1[, trend, method, ...])`   | 一変量方程式の共和分検定。                                               |
| `stattools.bds(x[, max_dim, epsilon, distance])`  | 時系列の独立性のためのBDS検定統計量。                                   |
| `stattools.q_stat(x, nobs)`                      | Ljung-Box Q 統計量を計算します。                                        |
| `stattools.breakvar_heteroskedasticity_test(resid)`| 残差の異方性を検定します。                                            |
| `stattools.grangercausalitytests(x, maxlag[, ...])`| 2つの時系列のGranger因果関係のための4つの検定。                    |
| `stattools.levinson_durbin(s[, nlags, isacov])`  | 自己回帰プロセスのためのLevinson-Durbin再帰。                        |
| `stattools.innovations_algo(acov[, nobs, rtol])`  | イノベーションアルゴリズムを使用して自己共分散をMAパラメータに変換します。|
| `stattools.innovations_filter(endog, theta)`     | イノベーションアルゴリズムを使用して観測値をフィルタリングします。       |
| `stattools.levinson_durbin_pacf(pacf[, nlags])`  | 自己相関関数（ACF）とAR係数を返すLevinson-Durbinアルゴリズム。        |
| `stattools.arma_order_select_ic(y[, max_ar, ...])`| 多くのARMAモデルの情報基準を計算します。                            |
| `x13.x13_arima_select_order(endog[, ...])`        | x12/x13 ARIMAを使用して自動季節性ARIMAオーダーの同定を実行します。     |
| `x13.x13_arima_analysis(endog[, maxorder, ...])`  | 月次または四半期データのx13-arima解析を実行します。                    |


### 推定

次のものは、statsmodels.tsa.apiを介してアクセスできる主要な推定クラスおよびその結果クラスです。
### 単変量自己回帰プロセス（AR）¶

Statsmodelsの基本的な自己回帰モデルは次のとおりです：

| 関数                                  | 説明                                                           |
|-------------------------------------|----------------------------------------------------------------|
| `ar_model.AutoReg(endog, lags[, trend, ...])`        | 自己回帰 AR-X(p) モデル                                       |
| `ar_model.AutoRegResults(model, params, ...)`        | AutoReg モデルの適合結果を保持するクラス                   |
| `ar_model.ar_select_order(endog, maxlag[, ...])`     | 自己回帰 AR-X(p) モデルのオーダー選択                     |


ar_model.AutoRegモデルは条件付きMLE（OLS）を使用してパラメータを推定し、外生的な回帰変数（AR-Xモデル）および季節効果をサポートします。

AR-Xおよび関連モデルは、arima.ARIMAクラスとSARIMAXクラス（Kalmanフィルタを介した完全なMLEを使用）でフィットすることもできます。

概要についてはノートブック「自己回帰」をご覧ください。
### 自己回帰移動平均プロセス（ARMA）およびカルマンフィルター¶

基本的なARIMAモデルと結果クラスは次のとおりです：

| 関数                                       | 説明                                                         |
|------------------------------------------|------------------------------------------------------------|
| `arima.model.ARIMA(endog[, exog, order, ...])`        | 自己回帰積分移動平均（ARIMA）モデルおよびその拡張版  |
| `arima.model.ARIMAResults(model, params, ...)`        | SARIMAXモデルを適合させた結果を保持するクラス         |


このモデルでは、さまざまな方法を使用してパラメータを推定できます（Hannan-Rissanen法を使用した条件付きMLEおよびKalmanフィルタを介した完全なMLEを含む）。これはSARIMAXモデルの特殊なケースであり、状態空間モデルから多くの機能を受け継いでいます（予測/予測、残差診断、シミュレーションおよびインパルス応答など）。

概要についてはノートブック「ARMA: Sunspots Data」および「ARMA: Artificial Data」をご覧ください。
### 指数平滑法¶

線形および非線形の指数平滑モデルが利用可能です。

| 関数                                     | 説明                                       |
|----------------------------------------|------------------------------------------|
| `ExponentialSmoothing(endog[, trend, ...])`        | Holt Winter's指数平滑                   |
| `SimpleExpSmoothing(endog[, ...])`               | 単純指数平滑                              |
| `Holt(endog[, exponential, damped_trend, ...])`   | Holt's指数平滑                           |
| `HoltWintersResults(model, params, sse, aic, ...)` | 指数平滑モデルの適合結果                  |


また、線形および非線形の指数平滑モデルは、通常のパラメータ適合、インサンプル予測、アウトオブサンプル予測のサポートに加えて、予測区間、シミュレーションなどもサポートするように、"イノベーション"の状態空間アプローチに基づいて実装されています。

| 関数                                    | 説明                                        |
|----------------------------------------|-------------------------------------------|
| `exponential_smoothing.ets.ETSModel(endog[, ...])`   | ETSモデル                               |
| `exponential_smoothing.ets.ETSResults(model, ...)`  | 誤差、トレンド、季節（ETS）指数平滑モデルの結果 |


最後に、線形指数平滑モデルは、一般的な状態空間フレームワークの特別なケースとしても別に実装されています（これは上記で説明した「イノベーション」な状態空間アプローチとは別です）。このアプローチは非線形（乗法）指数平滑モデルを許容しませんが、状態空間モデルのすべての機能（予測/予測、残差診断、シミュレーション、インパルス応答など）を含んでいます。

| 関数                                               | 説明                              |
|---------------------------------------------------|---------------------------------|
| `statespace.exponential_smoothing.ExponentialSmoothing(endog)` | 線形指数平滑モデル                  |
| `statespace.exponential_smoothing.ExponentialSmoothingResults(...)` | 線形指数平滑モデルの適合結果 |


概要については「指数平滑」のノートブックを参照してください。
### ARMAプロセス

以下は、与えられたラグ多項式の理論的な性質を扱うためのツールです。

| 関数                                     | 説明                               |
|-----------------------------------------|----------------------------------|
| `arima_process.ArmaProcess([ar, ma, nobs])` | 指定されたラグ多項式に対するARMAプロセスの理論的特性 |
| `arima_process.ar2arma(ar_des, p, q[, n, ...])` | ARプロセスのARMA近似を見つける |
| `arima_process.arma2ar(ar, ma[, lags])` | ARMAプロセスの有限ラグAR近似 |
| `arima_process.arma2ma(ar, ma[, lags])` | ARMAプロセスの有限ラグ近似MA表現 |
| `arima_process.arma_acf(ar, ma[, lags])` | ARMAプロセスの理論的自己相関関数 |
| `arima_process.arma_acovf(ar, ma[, nobs, ...])` | 定常ARMAプロセスの理論的自己共分散 |
| `arima_process.arma_generate_sample(ar, ma, ...)` | ARMAからデータをシミュレーション |
| `arima_process.arma_impulse_response(ar, ma)` | ARMAプロセスのインパルス応答関数（MA表現）を計算 |
| `arima_process.arma_pacf(ar, ma[, lags])` | ARMAプロセスの理論的偏自己相関関数 |
| `arima_process.arma_periodogram(ar, ma[, ...])` | ラグ多項式arとmaによるARMAプロセスのパワースペクトル |
| `arima_process.deconvolve(num, den[, n])` | 信号から除数を除去、n個の項の多項式の除算 |
| `arima_process.index2lpol(coeffs, index)` | 係数をラグ多項式に展開 |
| `arima_process.lpol2index(ar)` | ラグ多項式からゼロを削除 |
| `arima_process.lpol_fiar(d[, n])` | 分数積分のAR表現 |
| `arima_process.lpol_fima(d[, n])` | 分数積分のMA表現 |
| `arima_process.lpol_sdiff(s)` | 季節的な差分の係数を返す（1-L^s） |
| `ArmaFft(ar, ma, n)` | ARMAプロセス用のfftツール |


### 自己回帰分散ラグ（ARDL）モデル

自己回帰分散ラグモデルは、自己回帰モデル（AutoReg）とベクトル自己回帰モデル（VAR）の間をカバーしています。

| 関数                                     | 説明                              |
|-----------------------------------------|----------------------------------|
| `ardl.ARDL(endog, lags[, exog, order, trend, ...])` | Autoregressive Distributed Lag (ARDL)モデル |
| `ardl.ARDLResults(model, params, cov_params)` | ARDLモデルのフィッティング結果を保持するクラス |
| `ardl.ardl_select_order(endog, maxlag, exog, ...)` | ARDLモデルのオーダー選択 |
| `ardl.ARDLOrderSelectionResults(model, ics, ...)` | ARDLオーダー選択の結果 |


ardl.ARDLモデルは、条件付き最尤推定（OLS）を使用してパラメータを推定し、トレンドや季節のダミーを含む単純な決定論的要因と、DeterministicProcessを使用した複雑な決定論的要因の両方をサポートします。

AR-Xモデルおよび関連するモデルは、SARIMAXクラスを使用しても適合でき、カルマンフィルタを介した完全な最尤推定（MLE）が行われます。

詳細については「自己回帰分散ラグモデルの概要」ノートブックをご覧ください。

### 誤差修正モデル（ECM）

誤差修正モデルは、ARDLモデルの再パラメータ化で、内生変数のラグレベルと外生変数のラグ差の差を回帰するモデルです。

| 関数                                  | 説明                                         |
|--------------------------------------|---------------------------------------------|
| `ardl.UECM(endog, lags[, exog, order, trend, ...])` | Unconstrained Error Correlation Model (UECM) |
| `ardl.UECMResults(model, params, cov_params)` | UECMモデルのフィッティング結果を保持するクラス |
| `ardl.BoundsTestResult(stat, crit_vals, ...)` | 属性 |


### 状態空間モデル
https://www.statsmodels.org/stable/statespace.html#statespace

モデルと推定

以下は、主要な推定クラスで、statsmodels.tsa.statespace.apiを介してアクセスでき、それらの結果クラスです。

### 季節的な自己回帰統合移動平均と外生変数（SARIMAX）

SARIMAXクラスは、推定のためにstatespaceバックエンドを使用して作成されたモデルの典型的な例です。 SARIMAXは、tsaモデルと非常に類似して使用できますが、加法的および乗法的季節効果の推定、および任意のトレンド多項式の追加により、さまざまなモデルで動作します。

| 関数 | 説明 |
| --- | --- |
| `sarimax.SARIMAX(endog[, exog, order, ...])` | 季節性を考慮した外生変数を用いた季節性自己回帰和分移動平均（SARIMAX）モデルの定義 |
| `sarimax.SARIMAXResults(model, params, ...[, ...])` | SARIMAXモデルの適合結果を保持するクラス |


このモデルの使用例については、SARIMAXの使用例ノートブックまたは非常に簡単なコードスニペットを以下に示します：

```python
# statsmodelsのAPIをロード
import statsmodels.api as sm

# データセットを読み込む
endog = pd.read_csv('あなたの/データセット/ファイル.csv')

# AR(2)モデルを適合させることができます（上述の通り）
mod_ar2 = sm.tsa.SARIMAX(endog, order=(2, 0, 0))
# mod_ar2はSARIMAXクラスのインスタンスであることに注意

# 最大対数尤度法によってモデルを適合
res_ar2 = mod_ar2.fit()
# res_ar2はSARIMAXResultsクラスのインスタンスであることに注意

# 結果の概要を表示
print(res_ar2.summary())

# 季節成分を含むより複雑なモデルも適合できます。
# 例として、SARIMA(1,1,1) x (0,1,1,4)があります。
mod_sarimax = sm.tsa.SARIMAX(endog, order=(1, 1, 1),
                             seasonal_order=(0, 1, 1, 4))
res_sarimax = mod_sarimax.fit()

# 結果の概要を表示
print(res_sarimax.summary())
```
結果オブジェクトには、標準誤差、z-統計量、予測／予測など、他のstatsmodels結果オブジェクトから期待される多くの属性とメソッドが含まれています。

SARIMAXモデルは、モデル仕様に基づいてデザインおよび遷移行列（および場合によっては他の行列のいくつか）を生成します。 
### Unobserved Components
Unobserved Componentsクラスもまた、状態空間モデルの例です。

| 関数 | 説明 |
| --- | --- |
| `structural.UnobservedComponents(endog[, ...])` | 単変量未観測成分時系列モデルの定義 |
| `structural.UnobservedComponentsResults(...)` | 未観測成分モデルの適合結果を保持するクラス |


このモデルの使用例については、使用例ノートブックまたは時間系列データをトレンドとサイクルに分解するための無観測成分モデルを使用したノートブックを参照するか、以下の非常に簡単なコードスニペットをご覧ください：

```python
# statsmodelsのAPIをロード
import statsmodels.api as sm

# データセットを読み込む
endog = pd.read_csv('あなたの/データセット/ファイル.csv')

# ローカルレベルモデルを適合
mod_ll = sm.tsa.UnobservedComponents(endog, 'local level')
# mod_llはUnobservedComponentsクラスのインスタンスであることに注意

# 最大対数尤度法によってモデルを適合
res_ll = mod_ll.fit()
# res_llはUnobservedComponentsResultsクラスのインスタンスであることに注意

# 結果の概要を表示
print(res_ll.summary())

# 推定されたレベルとトレンド成分の系列のプロットを表示
fig_ll = res_ll.plot_components()

# 次に、次のようにダンピングされた確率的サイクルを追加できます
mod_cycle = sm.tsa.UnobservedComponents(endog, 'local level', cycle=True,
                                        damped_cycle=True,
                                        stochastic_cycle=True)
res_cycle = mod_cycle.fit()

# 結果の概要を表示
print(res_cycle.summary())

# 推定されたレベル、トレンド、およびサイクル成分の系列のプロットを表示
fig_cycle = res_cycle.plot_components()
```

### Vector Autoregressive Moving-Average with eXogenous regressors (VARMAX)¶

VARMAXクラスは、多変量状態空間モデルの例です。

| 関数 | 説明 |
| --- | --- |
| `varmax.VARMAX(endog[, exog, order, trend, ...])` | ベクトル自己回帰移動平均モデル（VARMAX）の定義 |
| `varmax.VARMAXResults(model, params, ...[, ...])` | VARMAXモデルの適合結果を保持するクラス |


このモデルの使用例については、VARMAXの使用例ノートブックを参照するか、以下の非常に簡単なコードスニペットをご覧ください：

```python
# statsmodelsのAPIをロード
import statsmodels.api as sm

# データセットを読み込む（多変量）
endog = pd.read_csv('あなたの/データセット/ファイル.csv')

# ローカルレベルモデルを適合
mod_var1 = sm.tsa.VARMAX(endog, order=(1,0))
# mod_var1はVARMAXクラスのインスタンスであることに注意

# 最大対数尤度法によってモデルを適合
res_var1 = mod_var1.fit()
# res_var1はVARMAXResultsクラスのインスタンスであることに注意

# 結果の概要を表示
print(res_var1.summary())

# インパルス応答を構築
irfs = res_var1.impulse_responses(steps=10)
```
### Dynamic Factor Models¶

Statsmodelsには、動的ファクターモデルをサポートする2つのクラスがあります：DynamicFactorMQおよびDynamicFactor。これらのモデルのそれぞれに利点がありますが、一般的にはDynamicFactorMQクラスがお勧めです。これは、Expectation-Maximization（EM）アルゴリズムを使用してパラメータを適合し、より堅牢で数百の観測系列を含めることができるためです。また、どの変数がどの要因にロードされるかのカスタマイズも可能です。ただし、まだ外生変数を含む機能はサポートしていません。一方、DynamicFactorはその機能をサポートしています。


| 関数 | 説明 |
| --- | --- |
| `dynamic_factor_mq.DynamicFactorMQ(endog[, ...])` | EMアルゴリズムを使用したダイナミックファクターモデル。月次/四半期データのオプションがあります。 |
| `dynamic_factor_mq.DynamicFactorMQResults(...)` | ダイナミックファクターモデルの適合結果を保持するクラス。 |


以下のコードは、DynamicFactorMQクラスを使用した簡単なコードスニペットの例です。

```python
# statsmodelsのAPIをロード
import statsmodels.api as sm

# データセットを読み込む
endog = pd.read_csv('あなたの/データセット/ファイル.csv')

# 動的ファクターモデルを作成
mod_dfm = sm.tsa.DynamicFactorMQ(endog, k_factors=1, factor_order=2)
# mod_dfmはDynamicFactorMQクラスのインスタンスであることに注意

# EMアルゴリズムを使用して最大対数尤度法によってモデルを適合
res_dfm = mod_dfm.fit()
# res_dfmはDynamicFactorMQResultsクラスのインスタンスであることに注意

# 結果の概要を表示
print(res_ll.summary())

# 内生変数に対する個々の推定要因の回帰のr^2値をプロットします
fig_dfm = res_ll.plot_coefficients_of_determination()
```

DynamicFactorクラスは、観測変数が少ないモデルに適しています。

| 関数 | 説明 |
| --- | --- |
| `dynamic_factor.DynamicFactor(endog, ...[, ...])` | ダイナミックファクターモデル |
| `dynamic_factor.DynamicFactorResults(model, ...)` | DynamicFactorモデルの適合結果を保持するクラス。 |


このモデルの使用例については、Dynamic Factorの例のノートブックを参照してください。

### 線形指数平滑モデル

ExponentialSmoothingクラスは、状態空間アプローチを使用して線形指数平滑モデルを実装したものです。

注意：このモデルはsm.tsa.statespace.ExponentialSmoothingで利用可能であり、sm.tsa.ExponentialSmoothingで利用可能なモデルとは異なります。これらのクラスの違いの詳細については、以下を参照してください。

| 関数 | 説明 |
| --- | --- |
| `exponential_smoothing.ExponentialSmoothing(endog)` | 線形指数平滑モデル |
| `exponential_smoothing.ExponentialSmoothingResults(...)` | 線形指数平滑モデルの適合結果を保持するクラス。 |


以下は非常に簡単なコードスニペットです：

```python
# statsmodels APIをロード
import statsmodels.api as sm

# データセットを読み込む
endog = pd.read_csv('your/dataset/here.csv')

# 単純指数平滑法、(A,N,N) で表される
mod_ses = sm.tsa.statespace.ExponentialSmoothing(endog)
res_ses = mod_ses.fit()

# ホルトの線形法、(A,A,N) で表される
mod_h = sm.tsa.statespace.ExponentialSmoothing(endog, trend=True)
res_h = mod_h.fit()

# 減衰トレンドモデル、(A,Ad,N) で表される
mod_dt = sm.tsa.statespace.ExponentialSmoothing(endog, trend=True, damped_trend=True)
res_dt = mod_dt.fit()

# ホルト・ウィンタースのトレンドと季節性法、(A,A,A) で表される
# (例えば、`endog`が季節的な周期性を持つ場合、四半期データなどの場合)
mod_hw = sm.tsa.statespace.ExponentialSmoothing(endog, trend=True, seasonal=4)
res_hw = mod_hw.fit()
```

Statsmodelsの指数平滑モデルクラス（`sm.tsa.statespace.ExponentialSmoothing`）と`sm.tsa.ExponentialSmoothing`モデルクラスの違いについて、以下の点に注意してください。

1. このモデルクラスは線形指数平滑モデルのみをサポートし、`sm.tsa.ExponentialSmoothing`は多重化モデルもサポートしています。
2. このモデルクラスは指数平滑モデルを状態空間形式に変換し、カルマンフィルタを適用して状態を推定しますが、`sm.tsa.ExponentialSmoothing`は指数平滑再帰に基づいています。これはいくつかの場合、このモデルクラスを使用してパラメータを推定する方が、`sm.tsa.ExponentialSmoothing`よりもやや遅くなることを意味することがあります。
3. このモデルクラスは、予測の信頼区間を生成できます（ガウス誤差を前提として）、`sm.tsa.ExponentialSmoothing`は信頼区間をサポートしていません。
4. このモデルクラスは、初期値を目的関数から除外することができ、多くの初期状態を推定する場合（季節性周期が大きい場合など）のパフォーマンスを向上させることができます。
5. このモデルクラスは診断や固定パラメータなど、状態空間モデルで使用可能な多くの高度な機能をサポートしています。

注意：このクラスは「複数のエラー源」（MSOE）の状態空間形式に基づいており、単一のエラー源（SSOE）の形式ではありません。
### カスタム状態空間モデル

状態空間モデルの真の強力さは、カスタムモデルの作成と推定を許容することです。通常、それは次の2つのクラスを拡張することによって行われます。これらのクラスは、推定および結果の出力のために、状態空間表現、カルマンフィルタリング、および最尤度適合機能をまとめたものです。

| 関数 | 説明 |
| --- | --- |
| `mlemodel.MLEModel(endog, k_states[, exog, ...])` | 最尤法推定のための状態空間モデル |
| `mlemodel.MLEResults(model, params, results)` | 状態空間モデルの適合結果を保持するクラス。 |
| `mlemodel.PredictionResults(model, ...[, ...])` | 最尤法モデルからの予測結果。 |


カスタム状態空間モデルの作成と推定の基本的な例については、Local Linear Trendの例のノートブックを参照してください。より高度な例については、MLEModelおよびMLEResultsを拡張したSARIMAXおよびSARIMAXResultsクラスのソースコードを参照してください。

簡単なケースでは、モデルはMLEModelクラスを使用して完全に構築できます。たとえば、上記のAR(2)モデルは、次のコードのみを使用して構築および推定できます：

```python
import numpy as np
from scipy.signal import lfilter
import statsmodels.api as sm

# 真のモデルパラメータ
nobs = int(1e3)
true_phi = np.r_[0.5, -0.2]
true_sigma = 1**0.5

# 時系列のシミュレーション
np.random.seed(1234)
disturbances = np.random.normal(0, true_sigma, size=(nobs,))
endog = lfilter([1], np.r_[1, -true_phi], disturbances)

# モデルの構築
class AR2(sm.tsa.statespace.MLEModel):
    def __init__(self, endog):
        # 状態空間モデルの初期化
        super(AR2, self).__init__(endog, k_states=2, k_posdef=1,
                                  initialization='stationary')

        # 状態空間表現の固定要素の設定
        self['design'] = [1, 0]
        self['transition'] = [[0, 0],
                                  [1, 0]]
        self['selection', 0, 0] = 1

    # パラメータのモデルへの導入方法の記述
    def update(self, params, transformed=True, **kwargs):
        params = super(AR2, self).update(params, transformed, **kwargs)

        self['transition', 0, :] = params[:2]
        self['state_cov', 0, 0] = params[2]

    # 開始パラメータとパラメータ名の指定
    @property
    def start_params(self):
        return [0,0,1]  # これらは非常に単純です

# モデルの作成および適合
mod = AR2(endog)
res = mod.fit()
print(res.summary())
```

これにより、次のサマリーテーブルが得られます。

```
                           Statespace Model Results
==============================================================================
Dep. Variable:                      y   No. Observations:                 1000
Model:                            AR2   Log Likelihood               -1389.437
Date:                Wed, 26 Oct 2016   AIC                           2784.874
Time:                        00:42:03   BIC                           2799.598
Sample:                             0   HQIC                          2790.470
                               - 1000
Covariance Type:                  opg
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
param.0        0.4395      0.030     14.730      0.000       0.381       0.498
param.1       -0.2055      0.032     -6.523      0.000      -0.267      -0.144
param.2        0.9425      0.042     22.413      0.000       0.860       1.025
===================================================================================
Ljung-Box (Q):                       24.25   Jarque-Bera (JB):                 0.22
Prob(Q):                              0.98   Prob(JB):                         0.90
Heteroskedasticity (H):               1.05   Skew:                            -0.04
Prob(H) (two-sided):                  0.66   Kurtosis:                         3.02
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
```

結果オブジェクトには、他のstatsmodelsの結果オブジェクトから期待される多くの属性とメソッドが含まれており、標準誤差、z-統計量、予測、予測などがサポートされています。

より高度な使用も可能で、パラメータ変換の指定や、情報提供のためのパラメータ名の指定などが含まれます。

### 使用の概要

すべての状態空間モデルは、通常のStatsmodelsのパターンに従います：

1. 入力データセットを使用してモデルのインスタンスを構築します。

2. モデルにパラメータを適用して（たとえば、fitを使用して）結果のインスタンスを構築します。

3. 結果のインスタンスと対話して、推定されたパラメータを調査し、残差の診断を行い、予測、シミュレーション、またはインパルス応答を生成します。

このパターンの例は次のとおりです：

```python
# 例のマクロ経済データセットを読み込む
dta = sm.datasets.macrodata.load_pandas().data
# 日付文字列またはタイムスタンプを使用して、関連付けられた周波数を持つインデックスを確認するために、
# データにインデックスを設定
dta.index = pd.date_range('1959Q1', '2009Q3', freq='QS')

# ステップ1: USインフレーションデータのSARIMAXモデルを構築
model = sm.tsa.SARIMAX(dta.infl, order=(4, 0, 0), trend='c')

# ステップ2: 最尤法によってモデルのパラメータを適合
results = model.fit()

# ステップ3: 結果を調査/使用

# - 推定結果をまとめた表を表示
print(results.summary())

# - 推定されたパラメータのみを表示
print(results.params)

# - 標準化された残差に基づく診断図を作成：
#   (1) 時系列グラフ
#   (2) ヒストグラム
#   (3) Q-Qプロット
#   (4) 自己相関図
results.plot_diagnostics()

# - 診断仮説検定を調査
# Jarque-Bera: [検定統計量、p値、歪度、尖度]
print(results.test_normality(method='jarquebera'))
# Goldfeld-Quandt型テスト: [検定統計量、p値]
print(results.test_heteroskedasticity(method='breakvar'))
# Ljung-Boxテスト: 各ラグに対する[検定統計量、p値]
print(results.test_serial_correlation(method='ljungbox'))

# - 次の4つの値を予測
print(results.forecast(4))

# - 2020Q4までの予測
print(results.forecast('2020Q4'))

# - 2005Q1から始まるサンプル内のダイナミックな予測をプロットし、
#   2010Q4までのアウトオブサンプル予測と90%信頼区間を生成
predict_results = results.get_prediction(start='2005Q1', end='2010Q4', dynamic=True)
predict_df = predict_results.summary_frame(alpha=0.10)
fig, ax = plt.subplots()
predict_df['mean'].plot(ax=ax)
ax.fill_between(predict_df.index, predict_df['mean_ci_lower'],
                predict_df['mean_ci_upper'], alpha=0.2)

# - サンプルの終了後に新しいデータの2年間をシミュレート
print(results.simulate(8, anchor='end'))

# - 2年間のインパルス応答
print(results.impulse_responses(8))
```

### 推定/フィルタリング/スムージングの基本的なメソッドと属性

状態空間モデルで最もよく使用されるメソッドは次のとおりです：

- fit: 最尤法を使用してパラメータを推定し、結果のオブジェクトを返します（このオブジェクトは、推定されたパラメータでカルマンフィルタリングおよびスムージングを実行しています）。これが最も一般的に使用されるメソッドです。

- smooth: カルマンフィルタリングおよびスムージングを実行した後の特定のパラメータに関連付けられた結果のオブジェクトを返します。

- loglike: 指定されたパラメータを使用してデータの対数尤度を計算します。

状態空間モデルの有用な属性には次のものがあります：

- param_names: モデルで使用されるパラメータの名前

- state_names: （観測されない）状態ベクトルの要素の名前

- start_params: 数値最大尤度最適化の開始値として使用される初期パラメータ推定

あまり使用されない他のメソッドには次のものがあります：

- filter: カルマンフィルタリングのみを実行した後の特定のパラメータに関連付けられた結果のオブジェクトを返します（スムージングは行いません）。

- simulation_smoother: シミュレーションスムージングを実行できるオブジェクトを返します。

### 出力および事後推定のメソッドと属性

一般的に使用されるメソッドには次のものがあります：

- summary: モデルの適合統計、推定されたパラメータ、およびその他の要約出力を提示する表を作成します。

- predict: インサンプルの予測とアウトオブサンプルの予測（ポイント推定のみ）を計算します。

- get_prediction: 信頼区間を含むインサンプルの予測とアウトオブサンプルの予測を計算します。

- forecast: アウトオブサンプルの予測（ポイント推定のみ）を計算します（これはpredictの便宜ラッパーです）。

- get_forecast: 信頼区間を含むアウトオブサンプルの予測を計算します（これはget_predictionの便宜ラッパーです）。

- simulate: 状態空間モデルに従って新しいデータをシミュレートします。

- impulse_responses: 状態空間モデルからのインパルス応答を計算します。

一般的に使用される属性には次のものがあります：

- params: 推定されたパラメータ

- bse: 推定されたパラメータの標準誤差

- pvalues: 推定されたパラメータに関連付けられたp値

- llf: 推定されたパラメータでのデータの対数尤度

- sse、mse、およびmae: 二乗誤差の合計、平均二乗誤差、および平均絶対誤差

- 情報基準（AIC、AICc、BIC、およびHqCを含む）

- fittedvalues: モデルからの適合値（これらは一歩先の予測です）

- resid: モデルからの残差（これらは一歩先の予測誤差です）

### 観測されない状態の推定と共分散

観測データに条件づけられた観測されない状態ベクトルの推定を計算することは有用です。これらは結果オブジェクトのstatesに含まれており、次の要素を含みます：

- states.filtered: フィルタリングされた（片側）状態ベクトルの推定。時間tにおける状態ベクトルの推定は、時間tまでを含む観測データに基づいています。

- states.smoothed: スムージングされた（両側）状態ベクトルの推定。時間tにおける状態ベクトルの推定は、サンプル内のすべての観測データに基づいています。

- states.filtered_cov: フィルタリングされた（片側）状態ベクトルの共分散

- states.smoothed_cov: スムージングされた（両側）状態ベクトルの共分散

これらの各要素はPandas DataFrameオブジェクトです。

例として、UnobservedComponents componentsクラスを使用して推定された「ローカルレベル+季節」モデルでは、時系列の基本的なレベルと季節的な動きの推定を取得できます。

```python
fig, axes = plt.subplots(3, 1, figsize=(8, 8))

# 月次の衣類の小売売上高を取得
from pandas_datareader.data import DataReader
clothing = DataReader('MRTSSM4481USN', 'fred', start='1992').asfreq('MS')['MRTSSM4481USN']

# ローカルレベル + 季節モデルを構築
model = sm.tsa.UnobservedComponents(clothing, 'llevel', seasonal=12)
results = model.fit()

# データ、レベル、季節成分をプロット
clothing.plot(ax=axes[0])
results.states.smoothed['level'].plot(ax=axes[1])
results.states.smoothed['seasonal'].plot(ax=axes[2])
```

### 残差診断

組み込みまたはカスタムのいずれかの状態空間モデルの推定後には、モデルが基本的な統計的仮定に適合しているかどうかを評価するのに役立つ3つの診断テストがあります。これらのテストは次のとおりです：

- test_normality
- test_heteroskedasticity
- test_serial_correlation

同様の目的で使用できるいくつかの標準的な回帰残差のプロットも利用可能です。これらはplot_diagnosticsコマンドを使用して生成できます。

推定されたパラメータを更新されたまたは異なるデータセットに適用する

結果オブジェクトから推定されたパラメータを新しいデータセットに適用するために使用できる3つのメソッドがあります：

- append: 現在のサンプルの末尾に続く追加の観測データを含む新しい結果オブジェクトを取得します（新しい結果オブジェクトには現在のサンプルと追加の観測データの両方が含まれます）。

- extend: 現在のサンプルの末尾に続く追加の観測データのための新しい結果オブジェクトを取得します（新しい結果オブジェクトには新しい観測データのみが含まれ、現在のサンプルは含まれません）。

- apply: 完全に異なるデータセット用の新しい結果オブジェクトを取得します。

時系列データの交差検証演習の1つは、トレーニングサンプル（時間tまでの観測値）を使用してモデルのパラメータをフィットし、その後テストサンプル（観測値t+1、t+2、...）を使用してモデルの適合性を評価することです。これは、applyまたはextendを使用して便利に行うことができます。以下の例では、extendメソッドを使用しています。

```python
# サンプルマクロ経済データセットを読み込む
dta = sm.datasets.macrodata.load_pandas().data
# 関連付けられた頻度のインデックスを持つことを確認し、
# 日付文字列またはタイムスタンプを使用して時期を参照できるようにする
dta.index = pd.date_range('1959Q1', '2009Q3', freq='QS')

# インフレーションデータをトレーニングデータとテストデータに分割
training_endog = dta['infl'].iloc[:-1]
test_endog = dta['infl'].iloc[-1:]

# インフレーションのためのSARIMAXモデルを適合
training_model = sm.tsa.SARIMAX(training_endog, order=(4, 0, 0))
training_results = training_model.fit()

# 結果をテスト観測データに拡張
test_results = training_results.extend(test_endog)

# トレーニングサンプルのみを使用して計算されたパラメータに基づく
# テストサンプルの二乗誤差の合計
print(test_results.sse)
```
### Understanding the Impact of Data Revisions
データの修正の影響を理解するために使用できるnewsメソッドを提供しています。

| 関数 | 説明 |
| --- | --- |
| `news.NewsResults(news_results, model, ...[, ...])` | データの修正およびnewsが関心のある変数の推定値に与える影響 |


### 追加のオプションとツール

すべての状態空間モデルには、以下のオプションとツールがあります：

いくつかのパラメータを固定して残りのパラメータを最尤推定する

fit_constrainedメソッドを使用して、いくつかのパラメータを既知の値に固定し、残りのパラメータを最尤推定することができます。例を以下に示します：

```python
# モデルを構築する
model = sm.tsa.SARIMAX(endog, order=(1, 0, 0))

# パラメータ名を調べるために次のコードを使用します：
print(model.param_names)

# AR(1)係数を固定してモデルを適合する：
results = model.fit_constrained({'ar.L1': 0.5})
```

また、fix_paramsコンテキストマネージャを使用することもできます：

```python
# モデルを構築する
model = sm.tsa.SARIMAX(endog, order=(1, 0, 0))

# コンテキストマネージャを使用してAR(1)係数を固定してモデルを適合します
with model.fix_params({'ar.L1': 0.5}):
    results = model.fit()
```

### 低メモリオプション

観測データセットが非常に大きい場合やモデルの状態ベクトルが高次元である場合（例：長期の季節的な効果を考慮する場合）、デフォルトのメモリ要件は大きすぎる場合があります。そのため、fit、filter、およびsmoothメソッドはオプションのlow_memory=True引数を受け入れ、メモリ要件をかなり削減し、モデルの適合を高速化できます。

low_memory=Trueを使用する場合、すべての結果オブジェクトが利用できなくなることに注意してください。ただし、残差診断、インサンプル（非ダイナミック）予測、およびアウトオブサンプルの予測は引き続き利用可能です。

### 低レベルの状態空間表現とカルマンフィルタリング

最尤推定はモデルの尤度関数を評価する必要があり、状態空間形式のモデルの尤度関数はカルマンフィルタを実行することによって副産物として評価されます。

MLEModelでカスタムモデルを作成することはほとんどの場合、MLEModelおよびMLEResultsを拡張することによって行われますが、それらのクラスの背後にあるスーパーストラクチャを理解することは有用です。

最尤推定には、モデルの尤度関数を評価する必要があり、状態空間モデルの尤度関数はカルマンフィルタを実行することの副産物として評価されます。

MLEModelを使用してカスタムモデルを作成する場合、サポートされるオプションとツールは以下の通りです。

- Representationクラス: このクラスは状態空間モデルの表現を定義します。状態空間行列（design、obs_interceptなど）を保持し、それらを操作することを可能にします。

- FrozenRepresentationクラス: このクラスは、状態空間表現の「スナップショット」を取ります。使用可能な属性の完全なリストについては、クラスのドキュメンテーションを参照してください。

| 関数 | 説明 |
| --- | --- |
| `representation.Representation(k_endog, k_states)` | 時系列プロセスの状態空間表現 |
| `representation.FrozenRepresentation(model)` | 固定された状態空間モデル |


KalmanFilterクラスは、フィルタリング機能を提供するRepresentationのサブクラスです。状態空間表現行列が構築されたら、filterメソッドを呼び出してFilterResultsインスタンスを生成できます。FilterResultsはFrozenRepresentationのサブクラスです。

FilterResultsクラスは、状態空間モデルの凍結された表現（design、transitionなどの行列、およびモデルの次元など）を保持するだけでなく、フィルタリング出力も保持します。これにはフィルタされた状態と対数尤度が含まれます（使用可能な結果の一覧についてはクラスのドキュメンテーションを参照してください）。また、predictメソッドを提供し、インサンプル予測またはアウトオブサンプルの予測を可能にします。predictメソッドも同様のメソッドで、信頼区間を含む追加の予測結果や予測結果を提供します。

| 関数 | 説明 |
| --- | --- |
| `kalman_filter.KalmanFilter(k_endog, k_states)` | Kalmanフィルターを使用した時系列プロセスの状態空間表現 |
| `kalman_filter.FilterResults(model)` | 状態空間モデルにKalmanフィルターを適用した結果 |
| `kalman_filter.PredictionResults(results, ...)` | 状態空間モデルのインサンプルおよびアウトサンプル予測の結果 |


KalmanSmootherクラスは、スムージング機能を提供するKalmanFilterのサブクラスです。状態空間表現行列が構築されたら、filterメソッドを呼び出してSmootherResultsインスタンスを生成できます。SmootherResultsはFilterResultsのサブクラスです。

SmootherResultsクラスはFilterResultsからのすべての出力を保持しますが、スムージング出力も含まれ、スムージングされた状態と対数尤度が含まれます（使用可能な結果の一覧についてはクラスのドキュメンテーションを参照してください）。時刻tにおける「フィルタリング」出力は時刻tまでの観測値に条件付けた推定値を指し、時刻tまでのデータセット全体に条件付けた推定値を指す「スムージング」出力です。

| 関数 | 説明 |
| --- | --- |
| `kalman_smoother.KalmanSmoother(k_endog, k_states)` | Kalmanフィルターおよびスムーサーを使用した時系列プロセスの状態空間表現 |
| `kalman_smoother.SmootherResults(model)` | 状態空間モデルにKalmanスムーサーおよび/またはフィルターを適用した結果 |


SimulationSmootherクラスは、KalmanSmootherのサブクラスであり、さらにシミュレーションとシミュレーションスムージングの機能を提供します。simulation_smootherメソッドを呼び出すことで、SimulationSmoothResultsインスタンスが生成されます。

SimulationSmoothResultsクラスには、state vectorの joint posterior から抽出するためのシミュレーションスムージングを実行するsimulateメソッドがあります。これは、Gibbsサンプリングを使用した状態空間モデルのベイジアン推定に役立ちます。

| 関数 | 説明 |
| --- | --- |
| `simulation_smoother.SimulationSmoother(...)` | Kalmanフィルターおよびスムーサー、およびシミュレーションスムーサーを使用した時系列プロセスの状態空間表現 |
| `simulation_smoother.SimulationSmoothResults(...)` | 状態空間モデルにKalmanスムーサーおよび/またはフィルターを適用した結果 |
| `cfa_simulation_smoother.CFASimulationSmoother(model)` | "Cholesky Factor Algorithm" (CFA) シミュレーションスムーサー |


### 状態空間モデルのツール
状態空間モデリングやSARIMAXクラスで使用されるさまざまなツールがあります。

| 関数 | 説明 |
| --- | --- |
| `tools.companion_matrix(polynomial)` | 連立行列を作成する |
| `tools.diff(series[, k_diff, ...])` | 時系列データを差分を取る |
| `tools.is_invertible(polynomial[, threshold])` | 多項式が反転可能かどうかを判定 |
| `tools.constrain_stationary_univariate(...)` | 最適化プロセスで使用される非制約パラメータを制約パラメータに変換 |
| `tools.unconstrain_stationary_univariate(...)` | 尤度評価で使用される制約パラメータを最適化プロセスで使用される非制約パラメータに変換 |
| `tools.constrain_stationary_multivariate(...)` | ベクトル自己回帰モデルのために、最適化プロセスで使用される非制約パラメータを制約パラメータに変換 |
| `tools.unconstrain_stationary_multivariate(...)` | ベクトル自己回帰モデルのために、尤度評価で使用される制約パラメータを最適化プロセスで使用される非制約パラメータに変換 |
| `tools.validate_matrix_shape(name, shape, ...)` | 時変行列の形状を検証 |
| `tools.validate_vector_shape(name, shape, ...)` | 時変ベクトルの形状を検証 |


元のUser Guidでは、Vector ARs and Vector Error Correction Modelsの章が続きますが、取り敢えずここで終わります。